# Quantum State Tomography

* **Author:** Christopher J. Wood (cjwood@us.ibm.com)
* **Last Updated:** November 8, 2018 

In [1]:
# Needed for functions
import numpy as np
import time

# Import QISKit classes
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.tools.qi.qi import state_fidelity

# Tomography functions
import sys, os
sys.path.append(os.path.abspath(os.path.join('../qiskit_ignis')))
import tomography as tomo

## 2-Qubit state tomography Example

In [2]:
# Create a state preparation circuit
q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.h(q2[0])
bell.cx(q2[0], q2[1])
print(bell.qasm())

job = qiskit.execute(bell, Aer.get_backend('statevector_simulator'))
psi_bell = job.result().get_data(bell).get('statevector')
print(psi_bell)

OPENQASM 2.0;
include "qelib1.inc";
qreg q0[2];
h q0[0];
cx q0[0],q0[1];

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [3]:
# Generate circuits and run on simulator
t = time.time()
qst_bell = tomo.state_tomography_circuits(bell, q2)
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)
print('Time taken:', time.time() - t)

# Extract tomography data so that countns are indexed by measurement configuration
# Note that the None labels are because this is state tomography instead of process tomography
# Process tomography would have the preparation state labels there

tomo_counts_bell = tomo.tomography_data(job.result(), qst_bell)
tomo_counts_bell

Time taken: 0.6353530883789062


{('X', 'X'): {'00': 2520, '11': 2480},
 ('X', 'Y'): {'00': 1249, '01': 1274, '10': 1217, '11': 1260},
 ('X', 'Z'): {'00': 1220, '01': 1298, '10': 1282, '11': 1200},
 ('Y', 'X'): {'00': 1232, '01': 1261, '10': 1237, '11': 1270},
 ('Y', 'Y'): {'01': 2524, '10': 2476},
 ('Y', 'Z'): {'00': 1261, '01': 1183, '10': 1292, '11': 1264},
 ('Z', 'X'): {'00': 1297, '01': 1268, '10': 1238, '11': 1197},
 ('Z', 'Y'): {'00': 1228, '01': 1279, '10': 1213, '11': 1280},
 ('Z', 'Z'): {'00': 2553, '11': 2447}}

In [4]:
# Generate fitter data and reconstruct density matrix
probs_bell, basis_matrix_bell = tomo.fitter_data(tomo_counts_bell)
rho_bell = tomo.state_cvx_fit(probs_bell, basis_matrix_bell)
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity =', F_bell)

Fit Fidelity = 0.9999389273659922


## Including Calibration Error (Simulated)

Suppose we include T1 relaxation during measurement, but all other circuit operations are perfect.
Noise parameters:
* measurement time: 5 microseconds
* T_1: 50 microseconds
* Population of excited state: 5%

The assignment fidelity for this can be calculated from
* P(0|0) = 1 - P(1|0)
* P(1|0) = pop1 * exp(- t_meas / t_1)
* P(0|1) = (1 - pop1) * exp(- t_meas / t_1)
* P(1|1) = 1 - P(0|1)

the calibration error matrix is
M = [[P(0|0), P(0|1)], [P(1|0), P(1|1)]]

In [5]:
pop1 = 0.05
T1 = 50
t_meas = 5

noise = {
    'relaxation_rate': 1. / T1,
    'thermal_populations': [1 - pop1, pop1],
    'measure': {'gate_time': t_meas}
}

# Generate circuits and run on simulator
job_cal = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000,
                         config={'noise_params': noise})

# Extract tomography data so that countns are indexed by measurement configuration
tomo_counts_noise = tomo.tomography_data(job_cal.result(), qst_bell)
tomo_counts_noise

{('X', 'X'): {'00': 2460, '01': 237, '10': 204, '11': 2099},
 ('X', 'Y'): {'00': 1498, '01': 1245, '10': 1218, '11': 1039},
 ('X', 'Z'): {'00': 1431, '01': 1296, '10': 1260, '11': 1013},
 ('Y', 'X'): {'00': 1472, '01': 1257, '10': 1228, '11': 1043},
 ('Y', 'Y'): {'00': 432, '01': 2292, '10': 2258, '11': 18},
 ('Y', 'Z'): {'00': 1444, '01': 1255, '10': 1263, '11': 1038},
 ('Z', 'X'): {'00': 1512, '01': 1230, '10': 1258, '11': 1000},
 ('Z', 'Y'): {'00': 1432, '01': 1206, '10': 1249, '11': 1113},
 ('Z', 'Z'): {'00': 2507, '01': 214, '10': 219, '11': 2060}}

In [6]:
# Create Calibration matrix

P1_0 = pop1 * np.exp(-t_meas / T1)
P0_1 = (1 - pop1) * np.exp(-t_meas / T1)

# Single qubit calibration matrix
cal_mat_single = np.array([[1 - P1_0, P0_1], [P1_0, 1 - P0_1]])

# 2-qubit calibration matrix
cal_mat = np.kron(cal_mat_single, cal_mat_single)

In [7]:
# Fitting without calibration matrix
probs_noise, basis_matrix_noise = tomo.fitter_data(tomo_counts_noise)
rho_noise = tomo.state_cvx_fit(probs_noise, basis_matrix_noise)
print('Fitting without using calibration data')
print('Fit Fidelity =', state_fidelity(psi_bell, rho_noise))

Fitting without using calibration data
Fit Fidelity = 0.8681015933385634


In [8]:
# Fitting with calibration matrix by modifying POVMs
probs_noise_cal, basis_matrix_noise_cal = tomo.fitter_data(tomo_counts_noise,
                                                     calibration_matrix=cal_mat)
rho_noise_cal = tomo.state_cvx_fit(probs_noise_cal, basis_matrix_noise_cal)
print('Using calibration data to update basis matrix')
print('Fit Fidelity =', state_fidelity(psi_bell, rho_noise_cal))

Using calibration data to update basis matrix
Fit Fidelity = 0.985520307246098


## Generating and fitting random states

We now test the functions on state generated by a circuit consiting of a layer of random single qubit unitaries u3

In [9]:
def random_u3_tomo(nq, shots):
    
    def rand_angles():
        return tuple(2 * np.pi * np.random.random(3) - np.pi)
    q = QuantumRegister(nq)
    circ = QuantumCircuit(q)
    for j in range(nq):
        circ.u3(*rand_angles(), q[j])
    job = qiskit.execute(circ, Aer.get_backend('statevector_simulator'))
    psi_rand = job.result().get_data(circ).get('statevector')
    
    qst_circs = tomo.state_tomography_circuits(circ, q)
    job = qiskit.execute(qst_circs, Aer.get_backend('qasm_simulator'),
                         shots=shots)
    tomo_counts = tomo.tomography_data(job.result(), qst_circs)
    probs, basis_matrix = tomo.fitter_data(tomo_counts, beta=0.5)
    
    rho_cvx = tomo.state_cvx_fit(probs, basis_matrix)
    rho_mle = tomo.state_mle_fit(probs, basis_matrix)
    
    print('F fit (CVX) =', state_fidelity(psi_rand, rho_cvx))
    print('F fit (MLE) =', state_fidelity(psi_rand, rho_mle))

In [10]:
for j in range(5):
    print('Random single-qubit unitaries: set {}'.format(j))
    random_u3_tomo(3, 5000)

Random single-qubit unitaries: set 0
F fit (CVX) = 0.9970702786442692
F fit (MLE) = 0.9943899714180672
Random single-qubit unitaries: set 1
F fit (CVX) = 0.9966216318400607
F fit (MLE) = 0.9933462944118627
Random single-qubit unitaries: set 2
F fit (CVX) = 0.997138676982347
F fit (MLE) = 0.9967221374053102
Random single-qubit unitaries: set 3
F fit (CVX) = 0.9999041896125411
F fit (MLE) = 0.9990113420414941
Random single-qubit unitaries: set 4
F fit (CVX) = 0.9997439765929161
F fit (MLE) = 0.9954484105588328


## 5-Qubit Bell State

In [11]:
# Create a state preparation circuit
q5 = QuantumRegister(5)
bell5 = QuantumCircuit(q5)
bell5.h(q5[0])
for j in range(4):
    bell5.cx(q5[j], q5[j + 1])

# Get ideal output state
job = qiskit.execute(bell5, Aer.get_backend('statevector_simulator'))
psi_bell5 = job.result().get_data(bell5).get('statevector')

# Generate circuits and run on simulator
t = time.time()
qst_bell5 = tomo.state_tomography_circuits(bell5, q5)
job = qiskit.execute(qst_bell5, Aer.get_backend('qasm_simulator'), shots=5000)

# Extract tomography data so that countns are indexed by measurement configuration
tomo_counts_bell5 = tomo.tomography_data(job.result(), qst_bell5)
probs_bell5, basis_matrix_bell5 = tomo.fitter_data(tomo_counts_bell5)
print('Time taken:', time.time() - t)

Time taken: 204.00656533241272


In [12]:
t = time.time()
rho_mle_bell5 = tomo.state_mle_fit(probs_bell5, basis_matrix_bell5)
print('MLE Reconstruction')
print('Time taken:', time.time() - t)
print('Fit Fidelity:', state_fidelity(psi_bell5, rho_mle_bell5))

MLE Reconstruction
Time taken: 3.31719708442688
Fit Fidelity: 0.9934583753879618


In [13]:
t = time.time()
rho_sdp_bell5 = tomo.state_cvx_fit(probs_bell5, basis_matrix_bell5, solver='CVXOPT')
print('SDP Reconstruction')
print('Time taken:', time.time() - t)
print('Fidelity:', state_fidelity(psi_bell5, rho_sdp_bell5))

SDP Reconstruction
Time taken: 48.56195878982544
Fidelity: 0.999928063990402
